<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">Learn by Doing LLM Projects</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Tracing a Medical Assistant RAG System chat with LangSmith</h2>
    
</div>

<div align="center">
  by <b>Pere Martra</b>
  <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>  
</div>

<div align="center">
<a target="_blank" href="https://www.langchain.com/langsmith"><img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/langsmith.png?raw=true"></a>  
</div>

In this notebook, we will explore how to trace the different calls that occur in a LangChain Agent using LangSmith. We will use a familiar agent, employed in the LangChain section of the course, where a RAG system with medical information was created.

So, not only will we observe the traces of the agent, but we will also examine the traces of the retriever. Additionally, we'll inspect the query sent to the vectorial database and the returned results.
__________
 *At this moment LangSmith is in closed Beta, you can request acces from the LangSmith webpage*: https://www.langchain.com/langsmith

#Installing libraries & Loading Dataset

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [1]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


In [2]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [3]:
#uncoment this line if you want to limit the size of the data.
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [4]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [5]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [6]:
df_document = df_loader.load()
display(df_document[:2])

[Document(page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.', metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}),
 Document(page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial 

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [7]:
from langchain.text_splitter import CharacterTextSplitter

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=1250,
                                      separator="\n",
                                      chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [9]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain.


### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [13]:
from utils import model_chooser
chooser= model_chooser()

In [14]:
from langchain_community.embeddings import OllamaEmbeddings

model_name = chooser.Model.value

embed = OllamaEmbeddings(
    model=model_name,
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [15]:
directory_cdb = './chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [17]:
from langchain_community.llms import Ollama
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm = Ollama(model=chooser.Model.value)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

We can try the isolated Retrieval to see if the information it returns is relevant.




In [18]:
qa.run("What is the main symptom of LCM?")

/work/06659/wmobley/ls6/miniconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The main symptom of Leptospirosis, also known as LCM, is diarrhea.'

In [21]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            """use this tool when answering medical knowledge queries to get
            more information about the topic"""
        )
    )
]

In [19]:
second_chooser = model_chooser()

In [24]:
from langchain.agents import create_react_agent
from langchain import hub

llm2 = Ollama(model=second_chooser.Model.value)


prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=llm2,
    prompt=prompt,
)

In [25]:
# Create an agent executor by passing in the agent and tools
from langchain.agents import AgentExecutor
agent_executor2 = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a request not related to the Medical field.

In [26]:
agent_executor2.invoke({"input": "What is 2 multiplied by 2?"})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? No
Final Answer: The answer to the multiplication problem "2 multiplied by 2" is 4.

> Finished chain.


{'input': 'What is 2 multiplied by 2?',
 'chat_history': [],
 'output': 'The answer to the multiplication problem "2 multiplied by 2" is 4.'}

The initial call to the Agent happens with just one call to OpenAI. One piece of information available in LangSmith is the entire prompt. I'll copy it just below the image, so you can see.




```
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

Medical KB: use this tool when answering medical knowledge queries to get
            more information about the topic

To use a tool, please use the following format:


Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [Medical KB]
Action Input: the input to the action
Observation: the result of the action


When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:


Thought: Do I need to use a tool? No
Final Answer: [your response here]


Begin!

Previous conversation history:
[]

New input: What is 2 multiplied by 2?
```

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [27]:
agent_executor2.memory.clear()

In [30]:
agent_executor2.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnosis?"""})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: BotulismThe answer to the question "Botulism" based on the provided context is:

Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by the bacterium Clostridium botulinum and sometimes by strains of Clostridium butyricum and Clostridium baratii. There are five main kinds of botulism: foodborne, wound, infant, adult intestinal colonization, and iatrogenic. All forms of botulism can be fatal and are considered medical emergencies. Treatment is available after accurate diagnosis, which typically involves prescription medication such as praziquantel or niclosamide taken by mouth. Based on the observation from using the Medical KB tool, the diagnosis of Botulism can be confirmed through the following steps:

1. Obtain a detailed medical history from the patient, including any recent consumption of contaminated food or exposure to environmental source

{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnosis?',
 'chat_history': [HumanMessage(content='Is this an important illness?'),
  AIMessage(content='The term "important illness" is quite subjective and can depend on various factors such as the specific condition being referred to, its impact on the individual\'s health and wellbeing, and other contextual factors. In general, any illness that requires medical attention or has the potential to cause significant harm or disability should be considered important. However, for a definitive answer, more information about the specific illness in question would be needed. If you have a particular condition in mind, please provide more details so I can assist you better.')],
 'output': 'The diagnosis of Botulism can be confirmed through obtaining a detailed medical history, performing a physical examination, requesting laboratory tests, and consulting with a healthcare professional.'}

In [29]:
agent_executor2.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? No
Final Answer: The term "important illness" is quite subjective and can depend on various factors such as the specific condition being referred to, its impact on the individual's health and wellbeing, and other contextual factors. In general, any illness that requires medical attention or has the potential to cause significant harm or disability should be considered important. However, for a definitive answer, more information about the specific illness in question would be needed. If you have a particular condition in mind, please provide more details so I can assist you better.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [],
 'output': 'The term "important illness" is quite subjective and can depend on various factors such as the specific condition being referred to, its impact on the individual\'s health and wellbeing, and other contextual factors. In general, any illness that requires medical attention or has the potential to cause significant harm or disability should be considered important. However, for a definitive answer, more information about the specific illness in question would be needed. If you have a particular condition in mind, please provide more details so I can assist you better.'}

On the left side of the image, you can see the various calls made by the Agent, which took 2.5 seconds to execute and consumed 2000 tokens.

I will try to describe what happens in each call.

**OpenAI**: he complete prompt, including the template from Hugging Face, and the user's question is passed to the OpenAI Model. It responds with the following step. Its answer is:

*Thought: Do I need to use a tool?*

*Yes  Action: Medical KB*  

*Action Input: Botulism*

**Medical KB**: The Agent utilizes the configured tool, passing only a single word as a parameter: "Botulism."

**Medical KB.Retriever:** The retriever returns four documents extracted from the Vectorial Database.

**Medical KB.OpenAI:** The prompt is constructed with the information from the vectorial database. Even though I won't include the paste, I manage to identify that the four returned documents are actually two but duplicated. This, in a real project, could have helped me detect some issue, perhaps I have duplicates in the dataset, or maybe I loaded them twice. In any case, I'm consuming many more tokens than necessary. The model returns a response created considering the information contained in the prompt.

**OpenAI:** In this final call to the model, it decides that the received response is what the user needs. Therefore, it marks it as correct and returns it to the user.


And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.


# Conclusions.
LangSmith is an incredibly useful tool for tracing and storing all the information generated when making calls to LangChain.

The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.



And you have all the information in a project stored in LangSmith!!!!


---